<a href="https://colab.research.google.com/github/VirtualGoat/Enhanced-CCTV-security-using-Deep-Learning/blob/master/Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install imageAI

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
def forFrame(frame_number, output_array, output_count, detected_copy):
    print("FOR FRAME " , frame_number)
#    print("Output for each object : ", output_array)
    print("Output count for unique objects : ", output_count)
    print("------------END OF A FRAME --------------")

def forSeconds(second_number, output_arrays, count_arrays, average_output_count, detected_copy):
    print("SECOND : ", second_number)
#    print("Array for the outputs of each frame ", output_arrays)
    print("Array for output count for unique objects in each frame : ", count_arrays)
    print("Output average count for unique objects in the last second: ", average_output_count)
    print("------------END OF A SECOND --------------")

def forMinute(minute_number, output_arrays, count_arrays, average_output_count, detected_copy):
    print("MINUTE : ", minute_number)
#    print("Array for the outputs of each frame ", output_arrays)
    print("Array for output count for unique objects in each frame : ", count_arrays)
    print("Output average count for unique objects in the last minute: ", average_output_count)
    print("------------END OF A MINUTE --------------")

In [0]:
yolo_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'
video_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'
execution_path = os.getcwd()
print(execution_path)


/content


In [0]:
pip install utils

In [0]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from utils import *
from DNModel import net as Darknet
from img_process import inp_to_image, custom_resize
import pandas as pd
import random 
import pickle as pkl
import argparse



def prepare_input(img, inp_dim):
    """
    Prepare image for inputting to the neural network. 
    Perform tranpose and return Tensor
    """

    orig_im = img
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (custom_resize(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

def write(x, img):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = random.choice(colors)
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

def arg_parse():
    """
    Parse arguements to the detect module
    
    """
    
    
    parser = argparse.ArgumentParser(description='YOLO v3 Video Detection Module')
   
    parser.add_argument("--video", dest = 'video', help = 
                        "Video to run detection upon",
                        default = "video.avi", type = str)
    parser.add_argument("--dataset", dest = "dataset", help = "Dataset on which the network has been trained", default = "pascal")
    parser.add_argument("--confidence", dest = "confidence", help = "Object Confidence to filter predictions", default = 0.5)
    parser.add_argument("--nms_thresh", dest = "nms_thresh", help = "NMS Threshhold", default = 0.4)
    parser.add_argument("--cfg", dest = 'cfgfile', help = 
                        "Config file",
                        default = "cfg/yolov3.cfg", type = str)
    parser.add_argument("--weights", dest = 'weightsfile', help = 
                        "weightsfile",
                        default = "yolov3.weights", type = str)
    parser.add_argument("--reso", dest = 'reso', help = 
                        "Input resolution of the network. Increase to increase accuracy. Decrease to increase speed",
                        default = "128", type = str)
    return parser.parse_args()


if __name__ == '__main__':
    args = arg_parse()
    confidence = float(args.confidence)
    nms_thesh = float(args.nms_thresh)
    start = 0

    CUDA = torch.cuda.is_available()

    num_classes = 80
    
    bbox_attrs = 5 + num_classes
    
    print("Loading network")
    model = Darknet(args.cfgfile)
    model.load_weights(args.weightsfile)
    print("Network loaded")
    classes = load_classes('data/coco.names')
    colors = pkl.load(open("pallete", "rb"))
    model.DNInfo["height"] = args.reso
    inp_dim = int(model.DNInfo["height"])


    if CUDA:
        model.cuda()
        
    model.eval()
    
    videofile = '/content/drive/My Drive/Colab Notebooks/ImageAI/messi.mp4'
    
    cap = cv2.VideoCapture(videofile)
    
    assert cap.isOpened(), 'Cannot capture source'
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        if ret:
            

            img, orig_im, dim = prepare_input(frame, inp_dim)
            
            im_dim = torch.FloatTensor(dim).repeat(1,2)                        
            
            
            if CUDA:
                im_dim = im_dim.cuda()
                img = img.cuda()
            
            with torch.no_grad():   
                output = model(Variable(img), CUDA)
            output = write_results(output, confidence, num_classes, nms = True, nms_conf = nms_thesh)

            if type(output) == int:
                cv2.imshow("frame", orig_im)
                key = cv2.waitKey(1)
                if key & 0xFF == ord('x'):
                    break
                continue
        
            
            im_dim = im_dim.repeat(output.size(0), 1)
            scaling_factor = torch.min(inp_dim/im_dim,1)[0].view(-1,1)
            
            output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
            output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2
            
            output[:,1:5] /= scaling_factor
    
            for i in range(output.shape[0]):
                output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
                output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])
            

            
            list(map(lambda x: write(x, orig_im), output))
            
            
            cv2.imshow("frame", orig_im)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('x'):
                break
        else:
            break
    

In [0]:
use_model = "yolo.h5"
use_video="cctv.mp4"
from imageai.Detection import VideoObjectDetection
import cv2
analisys_video = "detected_" + use_video

detector = VideoObjectDetection()
camera = cv2.VideoCapture("http://186.195.206.250:8080/view/index.shtml")
# Iniciate model
detector.setModelTypeAsYOLOv3()      # To YOLOv3 model
#detector.setModelTypeAsRetinaNet()   # To Retinanet model
#detector.setModelTypeAsTinyYOLOv3()   # To TinyYOLOv3 model

detector.setModelPath( os.path.join(yolo_path , use_model))
detector.loadModel()

vid_path = detector.detectObjectsFromVideo(camera_input=camera,
                                         # input_file_path=os.path.join( video_path, use_video),
                                           output_file_path=os.path.join(video_path, analisys_video),
                                           frames_per_second=30,
                                 #          per_second_function = forSeconds,
                                           per_frame_function = forFrame,
                                 #          per_minute_function= forMinute,
                                           minimum_percentage_probability=30,
                                           log_progress=True,
                                           return_detected_frame=True)
                                           

print(vid_path)

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive/')

In [0]:
import cv2 

vidObj = cv2.VideoCapture('/content/drive/My Drive/Colab Notebooks/ImageAI/messi1/messi.mp4') 

count = 0

success = 1

while success: 

  success, image = vidObj.read() 
  if count%20==0:
    cv2.imwrite("/content/drive/My Drive/Colab Notebooks/ImageAI/messi1/frame%d.jpg" % count, image) 

  count += 1


In [0]:
# from imageai.Detection import ObjectDetection
import os
use_model = "yolo.h5"
use_video="messi.mp4"
from imageai.Detection import VideoObjectDetection
import cv2
analisys_video = "detected_" + use_video

execution_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5"))
detector.loadModel()


video_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'


detections, objects_path = detector.detectObjectsFromVideo(
                                           input_file_path=os.path.join( video_path, use_video),
                                           output_file_path=os.path.join(video_path, analisys_video),
                                           frames_per_second=30,
                                 #          per_second_function = forSeconds,
                                           per_frame_function = forFrame,
                                 #          per_minute_function= forMinute,
                                           minimum_percentage_probability=30,
                                           log_progress=True,
                                           return_detected_frame=True,
                                           extract_detected_objects=True)

for eachObject, eachObjectPath in zip(detections, objects_path):
    print(eachObject["name"] , " : " , eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("Object's image saved in " + eachObjectPath)
    print("--------------------------------")

TypeError: ignored